# Meridonal Overturning Circulation
- calculations may be improved by taking partial bottom cells
- possibly SSH adds accuracy, but it is centered on the T-grid

In [ ]:
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - matplotlib.pyplot

In [ ]:
from MOC import calculate_MOC, AMOC_max, approx_lats
from grid import create_tdepth
from regions import Atlantic_mask, boolean_mask
from paths import file_ex_ocn_ctrl, path_results
from timeseries import IterateOutputCESM
from xr_DataArrays import xr_DZ, xr_DXU

In [ ]:
# %%time
# # ca 20 sec per file
# # 50 min for ctrl
# DXU  = xr_DXU('ocn')            # [m]
# DZU  = xr_DZ('ocn', grid='U')   # [m]
# # MASK = Atlantic_mask('ocn')     # Atlantic
# MASK = boolean_mask('ocn', 2)   # Pacific
# # MASK = boolean_mask('ocn', 0)   # Global OCean

# for run in ['ctrl']:#, 'rcp']:
#     for i, (y,m,s) in enumerate(IterateOutputCESM(domain='ocn', run=run, tavg='yrly', name='UVEL_VVEL')):
#         # ca. 20 sec per year
#         print(i, y, s)
#         ds = xr.open_dataset(s, decode_times=False)
#         MOC = calculate_MOC(ds=ds, DXU=DXU, DZU=DZU, MASK=MASK)
#         if i==0:
#             MOC_out = MOC.copy()
#         else:
#             MOC_out = xr.concat([MOC_out, MOC], dim='time')
#     #     if y==202: break
#     MOC_out.to_netcdf(f'{path_results}/MOC/PMOC_{run}.nc')

In [ ]:
AMOC_ctrl = xr.open_dataarray(f'{path_results}/MOC/AMOC_ctrl.nc', decode_times=False)
AMOC_rcp  = xr.open_dataarray(f'{path_results}/MOC/AMOC_rcp.nc' , decode_times=False)
PMOC_ctrl = xr.open_dataarray(f'{path_results}/MOC/PMOC_ctrl.nc', decode_times=False)
PMOC_rcp  = xr.open_dataarray(f'{path_results}/MOC/PMOC_rcp.nc' , decode_times=False)
GMOC_ctrl = xr.open_dataarray(f'{path_results}/MOC/GMOC_ctrl.nc', decode_times=False)
GMOC_rcp  = xr.open_dataarray(f'{path_results}/MOC/GMOC_rcp.nc' , decode_times=False)

In [ ]:
AMOC_ctrl.mean(dim='time').plot()

In [ ]:
PMOC_ctrl.mean(dim='time').plot()

In [ ]:
GMOC_ctrl.mean(dim='time').plot()

In [ ]:
AMOC_max(AMOC_rcp).plot()

In [ ]:
tdepth = create_tdepth('ocn')
lats = approx_lats('ocn')
X, Y = np.meshgrid(lats, -tdepth/1e3)


f, ax = plt.subplots(figsize=(8,5))
plt.tick_params(labelsize=14)

plt.pcolormesh(X, Y, AMOC_ctrl.mean(dim='time')/1e6, vmin=-20, vmax=20, cmap='RdBu_r')
plt.text(.05, .9, r'$\langle$CTRL$\rangle$', fontsize=16, transform=ax.transAxes)
plt.xlabel(r'latitude [$^\circ$N]', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
cbar = plt.colorbar(extend='max')
cbar.ax.set_ylabel('AMOC volume transport [Sv]', fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/MOC/AMOC_depth_lat_ctrl_mean')

    
f, ax = plt.subplots(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.pcolormesh(X, Y, (AMOC_rcp.mean(dim='time')-AMOC_ctrl.mean(dim='time'))/1e6, vmin=-2, vmax=2, cmap='RdBu_r')
plt.text(.05, .9, r'$\langle$RCP$\rangle$-$\langle$CTRL$\rangle$', fontsize=16, transform=ax.transAxes)
plt.xlabel(r'latitude [$^\circ$N]', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
cbar = plt.colorbar(extend='min')
cbar.ax.set_ylabel('AMOC volume transport [Sv]', fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/MOC/AMOC_depth_lat_rcp-ctrl')

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(8,5))
plt.tick_params(labelsize=14)
plt.plot(AMOC_ctrl.time/365     , AMOC_max(AMOC_ctrl)/1e6                                     , c='C0', lw=1)
plt.plot(AMOC_rcp .time/365-1800, AMOC_max(AMOC_rcp )/1e6                                     , c='C1', lw=1)
plt.plot(AMOC_ctrl.time/365     , AMOC_max(AMOC_ctrl).rolling(time=10, center=True).mean()/1e6, c='C0', lw=3, label='CTRL')
plt.plot(AMOC_rcp .time/365-1800, AMOC_max(AMOC_rcp ).rolling(time=10, center=True).mean()/1e6, c='C1', lw=3, label='RCP' )

plt.text(.98, .98, '26$^\circ$N, 1000 m', ha='right', va='top', fontsize=16, transform=ax.transAxes, color='grey')
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('AMOC strength [Sv]', fontsize=16)
plt.legend(fontsize=16, frameon=False)
plt.tight_layout()
plt.savefig(f'{path_results}/MOC/AMOC_timeseries')